### Imports

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
import os
import warnings

warnings.filterwarnings("ignore")

### Custom Imports

In [ ]:
sys.path.append(os.path.abspath('./PINNS/'))
sys.path.append(os.path.abspath('./PDE/'))
sys.path.append(os.path.abspath('./Losses/'))
from Poisson1D import Poisson1D
from Naive1D import PINN1D
from XPINN import XPINN
from VDLNet import VDLNet
from IPINN import IPINN
from Poisson1DLosses import Poisson1DLosses

In [ ]:
# Set the device to GPU if possible
torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

## One Dimensional Elliptic Interface Problem



We first take a 1D Poisson Equation as an example:
<br /> Domain: $Ω = [0, 1]$ 
<br /> Interface Points:  $x = [0.2,0.4,0.6,0.8]$
The governing PDE in each sub-domain $Ω_m$ (for $m = 1, 2, 3, 4, 5$), the boundary and interface conditions are

- $\frac{d}{dx}(κ_m\frac{du_m}{dx}) = -1$ in $Ω_m$,
- $u_1 = 0$ at $x = 0$ , $u_5 = 0$ at $x = 1$
- $[[u_k]] = 0$ at $x = 0.2, 0.4, 0.6, 0.8$
- $[[κ\frac{du}{dx}]] = 0$ at $x = 0.2, 0.4, 0.6, 0.8$

The material constants are given as $κ_1 = 1,κ_2 = 0.25, κ_3 = 0.9, κ_4 = 0.1$ and $κ_5 = 0.8$

In [ ]:
#define the global parameters
K = [1,0.25,0.9,0.1,0.8] 
gamma = [0,0.2,0.4,0.6,0.8,1] 
xs = np.arange(0,1,0.001)
torch_xs = torch.arange(0,1,0.001,device=device).reshape(-1,1)
num_pts = 2400
lsobj = Poisson1DLosses(n_points = num_pts,sub_domains = len(K),K = K,gamma = gamma)

In [ ]:
#Get the exact function
poisson1d = Poisson1D(K,gamma)
exact_val = poisson1d.equation(xs)
poisson1d.plot(100)

#### Traditional PINN

In [ ]:
#define the model
pinn = PINN1D(hidden_size = 10,depth = 3,act = nn.Tanh)
# Set the loss function
pinn.setLoss(lambda x: lsobj.loss(x,'pinn'))
#train the model
pinn.train(iterations = 1000)

In [ ]:
#Get the final results
pinn_pred = pinn.eval(torch_xs)
pinn_rmse = math.sqrt(np.mean(np.square(pinn_pred.reshape(-1,)-exact_val)))
pinn_l2 = math.sqrt(np.sum(np.square(pinn_pred.reshape(-1,)-exact_val))) / math.sqrt(np.sum(np.square(exact_val)))
print('RMSE: ',pinn_rmse, 'Relative L2 Error: ',pinn_l2)

### XPINN

We train the model after domain decomposition

In [ ]:
#define the model
xpinn = XPINN(num_domains=len(K),dimension = 1,hidden_size = 10,depth = 3,act = nn.Tanh)
# Set the loss function
lsobj.reset()
xpinn.setLoss(lambda x : lsobj.loss(x,'xpinn'))
#train the model
xpinn.train(iterations = 1000)

In [ ]:
outputs = xpinn.eval(torch_xs)
xpinn_pred = torch.zeros_like(outputs[0]).cpu().numpy().reshape(-1,)
for i in range(len(K)):
    ui = outputs[i].cpu().detach().numpy().reshape(-1,)
    mask = (xs >= gamma[i]) & (xs <= gamma[i+1]) 
    xpinn_pred[mask] = ui[mask]          
xpinn_rmse = math.sqrt(np.mean(np.square(xpinn_pred.reshape(-1,)-exact_val)))
xpinn_l2 = math.sqrt(np.sum(np.square(xpinn_pred.reshape(-1,)-exact_val))) / math.sqrt(np.sum(np.square(exact_val)))
print('RMSE: ',xpinn_rmse, 'Relative L2 Error: ',xpinn_l2)

### Vector Dense Layer

Each node in this NN gets its own activation funtion

In [ ]:
#define activations
act = [F.tanh,F.silu,F.relu,torch.sin,torch.cos]
#define the model
vdl = VDLNet(num_domains=len(K),activations = act,dimension = 1,hidden_size = 10,depth = 3)
# Set the loss function
lsobj.reset()
vdl.setLoss(lambda x : lsobj.loss(x,'xpinn'))
#train the model
vdl.train(iterations = 1000)

In [ ]:
outputs = vdl.eval(torch_xs)
vdl_pred = torch.zeros_like(outputs[0]).cpu().numpy().reshape(-1,)
for i in range(len(K)):
    ui = outputs[i].cpu().detach().numpy().reshape(-1,)
    mask = (xs >= gamma[i]) & (xs <= gamma[i+1]) 
    vdl_pred[mask] = ui[mask]          
vdl_rmse = math.sqrt(np.mean(np.square(vdl_pred.reshape(-1,)-exact_val)))
vdl_l2 = math.sqrt(np.sum(np.square(vdl_pred.reshape(-1,)-exact_val))) / math.sqrt(np.sum(np.square(exact_val)))
print('RMSE: ',vdl_rmse, 'Relative L2 Error: ',vdl_l2)

### IPINN
We decompose the domain but instead of making multiple models we create a single one with multiple activation functions based on sub-domain

In [ ]:
#define activation funtion
def cond_func(x,condition):
        if (condition == '0'):
            return F.silu(x)
        elif (condition == '1'):
            return F.tanh(x)
        elif (condition == '2'):
            return F.silu(x)
        elif(condition == '3'):
            return F.tanh(x)  
        elif(condition == '4'):
            return F.silu(x)

In [ ]:
#define the model
ipinn = IPINN(dimension=1,hidden_size=10,depth=3)
# Set the loss function
lsobj.reset()
ipinn.setLoss(lambda x : lsobj.loss(x,'ipinn',cond_func))
#train the model
ipinn.train(iterations = 1000)

In [ ]:
#Final Results
activations = [F.silu,F.tanh,F.silu,F.tanh,F.silu]
outputs = ipinn.eval(torch_xs,activations)
ipinn_pred = torch.zeros_like(outputs[0]).cpu().numpy().reshape(-1,)
for i in range(len(K)):
    ui = outputs[i].cpu().detach().numpy().reshape(-1,)
    mask = (xs >= gamma[i]) & (xs <= gamma[i+1]) 
    ipinn_pred[mask] = ui[mask]          
ipinn_rmse = math.sqrt(np.mean(np.square(ipinn_pred.reshape(-1,)-exact_val)))
ipinn_l2 = math.sqrt(np.sum(np.square(ipinn_pred.reshape(-1,)-exact_val))) / math.sqrt(np.sum(np.square(exact_val)))
print('RMSE: ',ipinn_rmse, 'Relative L2 Error: ',ipinn_l2)

### Results

In [ ]:
#Plots
plt.figure(figsize=(5,4))    
plt.plot(xs,exact_val,label = 'actual')
plt.plot(xs,pinn_pred,label = 'PINN')
plt.plot(xs,xpinn_pred,label = 'XPINN')
plt.plot(xs,vdl_pred,label = 'VDL')
plt.plot(xs,ipinn_pred,label = 'IPINN')
plt.ylim(bottom=0,top = 0.5)
plt.legend()
plt.title('1D-Poisson')
plt.show()

### Results
After running each model for an hour these were the results:

Metric|PINN|XPINN|XPINN(VDL)|IPINN
---|:---:|:---:|:---:|:---:
RMSE|$6.0 × 10^{-2}$|$6.6× 10^{-3}$|$4.9 × 10^{-3}$|$1.2 × 10^{-1}$  
Rel. L2|$2.8 \times 10^{-1}$|$3.1× 10^{-2}$|$2.3 × 10^{-2}$|$5.6 × 10^{-1}$   

All models had hidden layer size of $10$, depth of $3$ and were trained on $2400$ points

On Running Each model for 1000 iterations: 
Metric|PINN|XPINN|XPINN(VDL)|IPINN
---|:---:|:---:|:---:|:---:
RMSE|$1.1 × 10^{-1}$|$2.6× 10^{-2}$|$8.5 × 10^{-3}$|$1.0 × 10^{-1}$  
Rel. L2|$5.1 \times 10^{-1}$|$1.2 × 10^{-1}$|$4.0 × 10^{-2}$|$4.8 × 10^{-1}$
Time|39.7s|3m 27.9s|17m 49.6s|1m 6.0s      
